# Import package

In [1]:
import tensorflow as tf

import numpy as np

import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
# import seaborn as sns
#import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
import numpy as np
import PIL
#import requests
from io import BytesIO
from IPython.display import display
%matplotlib inline

#from mpl_toolkits.mplot3d import Axes3D
# from sklearn import datasets
# from sklearn.decomposition import PCA
# from sklearn.utils import shuffle

tf.enable_eager_execution()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation

import os
import scipy

# Import data

In [3]:
# Get the name of all the labels
home_path = '/home/bowen/Documents/Ecomed/'
data_path = '/home/bowen/Documents/Ecomed/dataset'
# class_names = os.listdir(data_path)
class_names = ["background", "pharmaceutical", "sharps", "trace_chemo"]

In [4]:
# Set up data generators that can read images from our dataset into Keras.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

data_generator = datagen.flow_from_directory(
        data_path,
        target_size=(224, 224),
        batch_size=8000,
        class_mode='binary')

Found 9003 images belonging to 4 classes.


# Load Model 

In [9]:
model_folder = '/home/bowen/Documents/Ecomed/models/model_4classes'
convert_model_folder = model_folder + '/convert'

## Callback Version

In [24]:
mobilenet = tf.keras.applications.MobileNet(include_top=False, input_shape=[224, 224, 3], weights='imagenet')
x = mobilenet.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
preds = tf.keras.layers.Dense(3, activation='softmax')(x)

# Combine feature layers with our new output layers.
model = tf.keras.Model(inputs=mobilenet.input, outputs=preds)
# defintion of optimizer
opt = tf.train.GradientDescentOptimizer(0.001)
# opt = tf.keras.optimizers.SGD(0.001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_path = os.path.join(model_folder, 'model_success.h5')
model.load_weights(model_path)

## OneSave Version

In [10]:
model_path = os.path.join(model_folder, "1551934800")
model = tf.contrib.saved_model.load_keras_model(model_path)

# Test Model 

In [11]:
with tf.device('/gpu:3'):
    result = model.evaluate_generator(data_generator)
print(model.metrics_names)
print(result)

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

# Convert Model

In [30]:
# Save tf.keras model in HDF5 format.
# keras_file = "/home/bowen/Documents/Ecomed/models/convert/model_success_middle.h5"
# tf.keras.models.save_model(model, keras_file)

In [34]:
# model_path = os.path.join('/home/bowen/Documents/Ecomed/models', '1551769465')
# model = tf.keras.models.load_model(model_path)

In [12]:
# Converting from keras is super simple, we simply point a tfliteconverter to our model!
converter = tf.contrib.lite.TFLiteConverter.from_saved_model(model_path)

INFO:tensorflow:Restoring parameters from /home/bowen/Documents/Ecomed/models/model_4classes/1551934800/variables/variables
INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default'}
INFO:tensorflow:input tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: input_2
INFO:tensorflow: tensor name: input_2:0, shape: (-1, 224, 224, 3), type: DT_FLOAT
INFO:tensorflow:output tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: dense_7
INFO:tensorflow: tensor name: dense_7/Softmax:0, shape: (-1, 4), type: DT_FLOAT
INFO:tensorflow:Restoring parameters from /home/bowen/Documents/Ecomed/models/model_4classes/1551934800/variables/variables
INFO:tensorflow:Froze 143 variables.
INFO:tensorflow:Converted 143 variables to const ops.


In [13]:
tflite_model = converter.convert()

In [15]:
# Specify where to save the new model.
tflite_model_path = os.path.join(convert_model_folder, 'model_4classes.tflite')
# Write the bytes of the model file to our drive.
open(tflite_model_path, "wb").write(tflite_model)
# Let's just check the size of the model.
print("Model size is %d KBytes." % (os.stat(tflite_model_path).st_size/1000))

Model size is 23300 KBytes.


# Test the tf.lite model

In [16]:
# Create a tflite interpreter, this is the part of tflite that actually runs models.
interpreter = tf.contrib.lite.Interpreter(model_path=tflite_model_path)
# Allocate memory for the all the weight tensors and such.
interpreter.allocate_tensors()
# Get input and output tensor information.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'input_2', 'index': 106, 'shape': array([  1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
[{'name': 'dense_7/Softmax', 'index': 102, 'shape': array([1, 4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]


In [20]:
with tf.device('/gpu:3'):
    def run_interpreter(input_data):
    #     interpreter.reset_all_variables()
        # Set tensor specifies the input to the model.
        interpreter.set_tensor(input_details[0]['index'], input_data)
        # Invoke is used to run the input through the model.
        interpreter.invoke()
        # We have to use get_tensor to load the output.
        output_data = interpreter.get_tensor(output_details[0]['index'])
        return np.argmax(output_data)

In [21]:
with tf.device('/gpu:3'):
    test_data, test_labels = data_generator.next()
    correct = 0
    total = 0
    for i, sample in enumerate(test_data):
        input_data = sample[tf.newaxis, ...]
        predict = run_interpreter(input_data)
        if predict == int(test_labels[i]):
            correct +=1
        total +=1
    print("Total Samples", total)
    print("accuracy", correct / total)

RuntimeError: There is at least 1 reference to internal data
      in the interpreter in the form of a numpy array or slice. Be sure to
      only hold the function returned from tensor() if you are using raw
      data access.